In [3]:
import numpy as np
import pandas as pd
# from bert_logistic import read_texts_from_dir
import os
def read_texts_from_dir(dir_path):
    """
    Reads the texts from a given directory and saves them in the pd.DataFrame with columns ['id', 'file_1', 'file_2'].

    Params:
      dir_path (str): path to the directory with data
    """
    # Count number of directories in the provided path
    dir_count = sum(
        os.path.isdir(os.path.join(root, d))
        for root, dirs, _ in os.walk(dir_path)
        for d in dirs
    )
    data = [0 for _ in range(dir_count)]
    print(f"Number of directories: {dir_count}")

    # For each directory, read both file_1.txt and file_2.txt and save results to the list
    i = 0
    for folder_name in sorted(os.listdir(dir_path)):
        folder_path = os.path.join(dir_path, folder_name)
        if os.path.isdir(folder_path):
            try:
                with open(
                    os.path.join(folder_path, "file_1.txt"), "r", encoding="utf-8"
                ) as f1:
                    text1 = f1.read().strip()
                with open(
                    os.path.join(folder_path, "file_2.txt"), "r", encoding="utf-8"
                ) as f2:
                    text2 = f2.read().strip()
                index = int(folder_name[-4:])
                data[i] = (index, text1, text2)
                i += 1
            except Exception as e:
                print(f"Error reading directory {folder_name}: {e}")

    # Change list with results into pandas DataFrame
    df = pd.DataFrame(data, columns=["id", "file_1", "file_2"]).set_index("id")
    return df

In [4]:
train_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/train"
test_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/test"
gt_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/train.csv"
print("Loading data...")
df_train = read_texts_from_dir(train_path)
df_test = read_texts_from_dir(test_path)
df_train_gt = pd.read_csv(gt_path)
y_train = df_train_gt["real_text_id"].values

Loading data...
Number of directories: 95
Number of directories: 1068


In [5]:
PROMPT = """You are an expert AI analyst specializing in identifying AI-generated textual anomalies. Your mission is to analyze a pair of scientific texts, text_0 and text_1, and determine which one is the "REAL" text (closely based on the original source) and which is the "FAKE" text (deliberately corrupted by an LLM).

Your primary goal is to identify the REAL text based on a hierarchical analysis of structural integrity, content coherence, and stylistic consistency.

**Decision-Making Heuristics & Key Differentiators:**

First, evaluate each text against these strong indicators of FAKE content. The presence of even one of these is a significant red flag.

**1. Structural & Statistical Anomalies (Highest Priority):**
    *   **Extreme Word Count:** FAKE texts are often either extremely short (e.g., under 100 words, sometimes empty) or excessively long (e.g., over 600 words). REAL texts consistently fall within a stable range, typically 150-500 words.
    *   **Text Corruption & Nonsensical Characters:** FAKE texts frequently contain a high ratio of non-Latin characters, Unicode gibberish (e.g., `\u0430\u043d\u0430 headers \u043c\u0430\u0440\u6e38`), or malformed code snippets.
    *   **Nonsensical Word Combinations:** Look for phrases that are grammatically plausible but semantically meaningless, like `"resolvediaz hen"` or `"eks empty"`. This indicates a failure in coherent text generation.

**2. Content & Tone Anomalies:**
    *   **Fantastical Content:** FAKE texts may abruptly introduce bizarre, non-scientific elements completely unrelated to the topic (e.g., Santa Claus, unicorns, galactic wars).
    *   **Inappropriate Tone Shift:** The text may suddenly shift from a formal, academic tone to an overly informal, narrative, or marketing style.

**Profile of a REAL Text:**
If a text does NOT exhibit any of the anomalies above, it is likely the REAL one. REAL texts are characterized by:
    *   **Structural Integrity:** Consistent and appropriate length (150-500 words).
    *   **Coherence and Focus:** The content is logical, on-topic, and uses specific, academic terminology relevant to astronomy or physics.
    *   **Cleanliness:** The text is free of random characters and nonsensical phrases.

**Your Step-by-Step Analysis Process:**

1.  **Structural Check:** Quickly assess `text_0` and `text_1` for word count and the presence of any obvious text corruption. This is your most reliable first filter.
2.  **Content Check:** Analyze both texts for fantastical content, tone shifts, and meaningless phrases.
3.  **Final Decision:** The text that shows structural integrity and content coherence is the REAL one. The other, exhibiting one or more anomalies, is the FAKE one.

**Input Texts:**

<text_0>
{text_0}
</text_0>

<text_1>
{text_1}
</text_1>

**Output Format:**
Provide your response as a single JSON object. Do not write any text outside of the JSON block. The JSON object must have the following structure:
{{
"label": 0
}}

If you determine text_0 is REAL, the label should be 0. If text_1 is REAL, the label should be 1.

Now, perform your analysis and return the JSON output.
"""

In [6]:
from pydantic import BaseModel, Field
class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    label: str = Field(description="If you determine text_0 is REAL, the label should be 0. If text_1 is REAL, the label should be 1.")

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.0,
    api_key=os.getenv("GEMINI_API_KEY")
)

llm_structured_output = llm.with_structured_output(ResponseFormatter)

In [28]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o",
    # temperature=0.0,
    api_key=os.getenv("OPENAI_API_KEY")
)

llm_structured_output = llm.with_structured_output(ResponseFormatter)

In [29]:
text_1 = df_train.loc[0]["file_1"]
text_2 = df_train.loc[0]["file_2"]

test_prompt = PROMPT.format(
    text_0=text_1,
    text_1=text_2
)

In [ ]:
sesponse = llm_structured_output.invoke(test_prompt)
response

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-svcac*************************************************************************************************************************************************dcAA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [10]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import json
import os

from dotenv import load_dotenv
# from google import genai
# from google.genai import types

load_dotenv()


def generate(prompt):
    response = llm_structured_output.invoke(prompt)
    return int(response.label)

In [11]:
predicted_label = []

In [12]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from tqdm import tqdm

# Reset predicted_label since the previous run failed
predicted_label = []

# Function to process a single row with retry logic
def process_single_row(row_data):
    file_1, file_2, row_id = row_data
    max_retries = 3
    
    for attempt in range(max_retries):
        try:
            prompt = PROMPT.format(text_0=file_1, text_1=file_2)
            response = generate(prompt)
            return row_id, response
        except Exception as e:
            if "rate_limit" in str(e).lower() or "quota" in str(e).lower():
                wait_time = (2 ** attempt) * 60  # Exponential backoff: 1min, 2min, 4min
                print(f"Rate limit hit for row {row_id}, waiting {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"Error processing row {row_id}: {e}")
                break
    
    return row_id, None  # Return None if all retries failed

# Prepare data for processing
row_data_list = [(row.file_1, row.file_2, row.Index) for row in df_test.itertuples()]

# Use ThreadPoolExecutor with limited workers to avoid overwhelming the API
results = {}
with ThreadPoolExecutor(max_workers=2) as executor:  # Reduced workers to avoid rate limits
    # Submit all tasks
    future_to_row = {executor.submit(process_single_row, row_data): row_data[2] 
                     for row_data in row_data_list}
    
    # Process completed tasks with progress bar
    for future in tqdm(as_completed(future_to_row), total=len(future_to_row), desc="Processing rows"):
        try:
            row_id, result = future.result()
            if result is not None:
                results[row_id] = result
            else:
                print(f"Failed to process row {row_id}")
        except Exception as e:
            row_id = future_to_row[future]
            print(f"Exception for row {row_id}: {e}")

# Convert results to list in correct order
predicted_label = [results.get(row_id, 0) for row_id in df_test.index]

print(f"Successfully processed {len([r for r in predicted_label if r != 0])}/{len(df_test)} rows")

Processing rows:   0%|          | 0/1068 [00:00<?, ?it/s]

Processing rows: 100%|██████████| 1068/1068 [1:27:22<00:00,  4.91s/it]

Successfully processed 443/1068 rows


In [13]:
len(predicted_label)

1068

In [14]:
predicted_label = np.array(predicted_label) + 1

In [ ]:
from pathlib import Path

print("Predicting on test ...")
test_pred = predicted_label

# --- Build submission -------------------------------------------------
submission = pd.DataFrame({
    "id": df_test.index,
    "real_text_id": test_pred.astype(int)
}).sort_values("id")

save_path = Path("submission_api_call_full_4o_new.csv")
submission.to_csv(save_path, index=False)
print(f"✅ Submission saved to {save_path.resolve()}")

Predicting on test ...
✅ Submission saved to /home/thangquang09/CODE/CTAI_MachineLearning/notebooks/thangquang/submission_api_call_full_gemini25pro_new.csv
